In [3]:
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import mandelbrot_matrix

reload(mandelbrot_matrix)

plt.style.use('seaborn')

re_lim=(-2,1) 
im_lim=(-1,1)

In [38]:
def generate_latinHyperCube():


    width_re = re_lim[1] - re_lim[0]
    width_im = im_lim[1] - im_lim[0]

    #print(width_re)
    #print(width_im)

    re_all = []
    im_all = []

    n = 1000
    factor_re = width_re/n
    factor_im = width_im/n

    low_re = re_lim[0]
    high_re = low_re + factor_re


    low_im = im_lim[0]
    high_im = low_im + factor_im

    for i in range(n):
        re = np.random.uniform(low_re, high_re) # type: ignore
        im = np.random.uniform(low_im,high_im) # type: ignore
        low_re = high_re
        high_re = low_re + factor_re
        low_im = high_im
        high_im = low_im + factor_im
        re_all.append(re)
        im_all.append(im)
    
    print(im_all)
    start = 0
    while(start+1 < n):
        selectIndex = np.random.randint(start,n)
        #swap start and selectIndex
        temp = im_all[selectIndex]
        im_all[selectIndex] = im_all[start]
        im_all[start] = temp
        start = start+1

    re_all = np.array(re_all)
    im_all = np.array(im_all)
    #print(im_all.shape)
    random_points = re_all+im_all*1j 
    
    return random_points

    #plt.plot(re_all, im_all, "o")    
    #plt.show()


$ \bar{X}_{j+1} = \bar{X}_{j} + \frac{X_{j+1} - \bar{X}_{j}}{j+1} $

$ {S}_{j+1}^{2} = (1-\frac{1}{j})S_{j}^{2}+(j+1)(\bar{X}_{j+1} - \bar{X}_{j})^{2} $

In [ ]:
%%time

n_sim = 100    # number of simulations to run
n_size = 1000   # number of ramdom points
threshold = 1000 # threshold for mandelbrot
data = []

for sim in range(n_sim):
    
    random_points = generate_latinHyperCube()
    
    # create the counter matrix "n" and the z matrix
    n = np.zeros(n_size)
    z = np.zeros(n_size)

    MAX_ITER = threshold
    # iterate until the threshold is reached
    for k in range(MAX_ITER):
        # add one to n_ij only if abs(z_ij[k])<=2
        n = np.where(np.abs(z) <= 2, n+1, n)
        # calculate z_ij[k+1] only if abs(z_ij[k])<=2
        z = np.where(np.abs(z) <= 2, np.square(z) + random_points, z)
                
    #print(z)
    count_inside = np.size(np.where(n>=threshold))
    #print(count_inside)
    
    # monte carlo
    total_points = n_size
    avg = count_inside/total_points
    
    
    
    data.append(avg)
    #print(avg)
    # average will follow a normal distribution for large values of n i.e. sim
    # alpha = 0.5
    # alpha/2 = 0.25. Z(alpha/2) = 1.96
    # percent confidence interval  = 100*(1-alpha) = 100*(1-0.5) = 95%
    # stop when 2*z*s/(sqrt(sim)) < l. 
    # if we just allow 1% error then 1% of 1.506484 = 0.01506484 ~ 0.015. Therefore our l is 0.015/Area of rectangle
    # l = (0.015)/6 = 0.0025
    # sample mean of (j+1)
    # Our samples - averages
    

prev_estimated_mean = np.sum(data)/(n_sim)
print(prev_estimated_mean)

mean_difference = 0

for i in range(n_sim):
    mean_difference += (data[i]-prev_estimated_mean)**2
    
prev_estimated_variance = mean_difference/(n_sim-1)



l = 0.0025
z = 1.96
estimated_std_deviation = np.sqrt(prev_estimated_variance)
acceptable_window = 2*z*estimated_std_deviation/np.sqrt(n_sim)
#print(acceptable_window)


while(acceptable_window > l):
    
        re, im, n = mandelbrot_matrix.random_mandelbrot_points(n_size, re_lim, im_lim,threshold=threshold)
        
        count_inside = np.size(np.where(n>=threshold))
    
        # monte carlo
        total_points = n_size
        avg = count_inside/total_points
        #data.append(avg)
        
        
        
        new_estimated_mean = prev_estimated_mean + (avg - prev_estimated_mean)/(n_sim+1)
        new_estimated_variance = (1-1/(n_sim))*prev_estimated_variance + (n_sim+1)*(new_estimated_mean-prev_estimated_mean)**2
        
        
        n_sim +=1
        #print(n_sim)
        prev_estimated_variance = new_estimated_variance
        prev_estimated_mean = new_estimated_mean
        
        estimated_std_deviation = np.sqrt(new_estimated_variance)
        acceptable_window = 2*z*estimated_std_deviation/np.sqrt(n_sim)
        #print(acceptable_window)




In [ ]:
print(n_sim)
# less simulations required as compared to pure random sampling